In [1]:
import sys
import os
import shutil 
from os.path import join

sys.path.append("/home/timssh/ML/TAGGING/CLS")
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from classification.train.model import InferDataset
from torch.utils.data import DataLoader
from classification.train.augmentation import PreProcess, DataAugmentation
import torch
from pathlib import Path

In [2]:
GROUP = "tits_size"
MODEL_ROOT = "/home/timssh/ML/TAGGING/DATA/models"

folder_ret = '/home/timssh/ML/TAGGING/GROP_AGR/tits_size'

In [3]:
Aug = DataAugmentation().eval()
Pre = PreProcess(gray=False, vflip=False, arch="eff")
# Load with extra files.
extra_files = {"num2label.txt": ""}  # values will be replaced with data
model = torch.jit.load(
    f"{MODEL_ROOT}/{GROUP}/v__2_all_eff_36_0.001/checkpoints/epoch=59-step=38280.pt",
    "cuda",
    _extra_files=extra_files,
)
model.eval()
model.to(torch.float16)
num2label = eval(extra_files["num2label.txt"])
print(num2label)

{'0': 'big tits', '1': 'flat chested', '2': 'huge tits', '3': 'small tits'}


In [4]:
for key, value in num2label.items():
    Path(join(folder_ret, value)).mkdir(parents=True, exist_ok=True)
Path(join(folder_ret, 'trash')).mkdir(parents=True, exist_ok=True)

In [5]:
masks = "/home/timssh/ML/TAGGING/DATA/segmentation/picture"
list_of_pics = [join(masks, path) for path in os.listdir(masks)]
dict_of_pics = {mask.split('.')[0] : mask for mask in os.listdir(masks)}
infer = InferDataset(sorted(list_of_pics), Pre)
samplt = infer[0]
loader = DataLoader(infer, 32, num_workers=32, pin_memory=True, shuffle=False)

In [6]:
for batch in loader:
    with torch.no_grad():
        ret_ = torch.round(
            torch.sigmoid(model(Aug(batch[0].to("cuda")))), decimals=2
        )
        ret_ = ret_.to("cpu")
        val, idx = ret_.max(axis=1)
        # print(val, idx)
        for num, id_ in enumerate(batch[1]):
            if val[num] > 0.9:
                tag = num2label[str(int(idx[num]))]
            else:
                tag = "trash"
            old_path = join(masks, dict_of_pics[id_])
            new_path = join(folder_ret, tag, dict_of_pics[id_])
            shutil.copyfile(old_path, new_path)

# TEST

In [14]:
from torch.utils.data import DataLoader, WeightedRandomSampler
import torch
import json
import pandas as pd
import numpy as np

In [7]:
with open('/home/timssh/ML/TAGGING/DATA/datasets/sex_positions.json', 'r') as json_file:
    json_ = json.load(json_file)
    num2label = json_["num2label"]
    weights = json_["weights"]
    data = json_['data']


In [8]:
df = pd.read_json(json_["data"])
df

,path,69,cowgirl,doggy style,missionary,reverse cowgirl,spooning
0,c65e5cc8ce4b232c3b39411b09fc57f0.jpeg,1,0,0,0,0,0
1,80279792ae8f037bc51e1f2386487998.jpeg,1,0,0,0,0,0
2,29e94c8fc081153e6681c96a4df7a63f.jpeg,1,0,0,0,0,0
3,25e87340b207e071bbcfa3618599a00f.jpeg,1,0,0,0,0,0
4,8a4b2f38811cd3a606ed929b2a6ff0cb.jpeg,1,0,0,0,0,0
...,...,...,...,...,...,...,...
27776,e6efbc00845bca88ee25f84b11a70fcc.jpeg,0,0,0,0,0,1
27777,ee223f3f6c90a675b680e4b640d445d3.jpeg,0,0,0,0,0,1
27778,867ee36efa23c356732ad9745b51d150.jpeg,0,0,0,0,0,1
27779,9618b6f63230d42764f37f01d5cd13d1.jpeg,0,0,0,0,0,1


In [26]:
cols = df.columns[1:]
df_len = len(df)
my_weights = torch.tensor([0.] * df_len)
all_ = 0
for col in cols:
    ret = df[col][df[col]==1].sum()
    print(ret)
    indexes_of_cls = list(df[col][df[col]==1].index)
    # for index in indexes_of_cls:
    my_weights[indexes_of_cls] = 1 / len(indexes_of_cls)
    all_ += ret
my_weights[my_weights == 0] = 1 / (df_len - all_)
print(all_)

1799
2172
2917
1831
2183
3395
14297


In [28]:
my_weights.sum()

tensor(7.0000)

In [29]:
df[df.columns[1:]].value_counts()

69  cowgirl  doggy style  missionary  reverse cowgirl  spooning
0   0        0            0           0                0           13484
                                                       1            3395
             1            0           0                0            2917
             0            0           1                0            2183
    1        0            0           0                0            2172
    0        0            1           0                0            1831
1   0        0            0           0                0            1799
dtype: int64